In [1]:
import tensorflow as tf
import numpy as np
from layers import architecture

# Use ADAM?
# Use better initializations?

OHE y

In [ ]:
inputs = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 16])
random_rolls = tf.placeholder(dtype=tf.float32, shape=[54])
weights = [var for var in tf.trainable_variables() if 'weights' in var.name]

lr = 0.1
weight_decay = 1e-4
momentum = 0.9
batch_size = 128
train_size = 45000

# both will have different graph structure, but will share weights.
with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
    out = architecture(inputs, random_rolls, True)
    val_out = architecture(input_vals, random_rolls, False)

val_acc = 100 * tf.reduce_sum(tf.argmax(tf.nn.softmax(val_out), axis=1) - tf.constant(y_val)) / len(y_val)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out,
                                                              labels=labels)) + 
       weight_decay * tf.add_n([tf.nn.l2_loss(weight) for weight in weights])
step = tf.train.MomentumOptimizer(lr, momentum, use_nesterov=True).minimize(loss)

In [6]:
with tf.Session() as sess: 
    merge = tf.summary.merge_all()
    writer = tf.summary.FileWriter("log", sess.graph)
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    start_index = 0
    iter_number = -1
    for epoch in range(500):
        if epoch in (250, 375):
            lr /= 10
        for _ in range(train_size // batch_size):
            iter_number += 1
            if start_index + batch_size >= train_size:
                diff = start_index + batch_size - train_size
                indices = list(range(start_index, train_size)) + list(range(diff))
                start_index = diff
            else:
                indices = list(range(start_index, start_index + batch_size))
                start_index += batch_size

            X_train_batch, y_train_batch = X_train[indices], y_train[indices]
            y_train_labels = tf.one_hot(y_train_batch, 10)
            # data augmentation
            x_offset = np.random.randint(-4, 4)
            y_offset = np.random.randint(-4, 4)
            flip_bool = np.random.uniform() > .5
            X_train_batch = np.roll(X_train_batch, (x_offset, y_offset), axis=(1, 2))
            if flip_bool:
                X_train_batch = np.flip(X_train_batch, axis=2)

            random_rolls_batch = np.random.uniform(size=54)
            _, loss_val = sess.run([step, loss], feed_dict={inputs: X_train_batch,
                                                            random_rolls: random_rolls_batch,
                                                            labels: y_train_labels})
            if iter_number % 100 == 0:
                val, merge_result = sess.run([val_acc, merge], feed_dict={input_vals: X_val,
                                                                          random_rolls: random_rolls_batch}) # random_rolls is irrelevant
                writer.add_summary(merge_result, iter_number)
        
            
    writer.close()

# Tests (ignore)

In [7]:
# # confirm tensorboard
# random_rolls_batch = np.random.uniform(size=54)
# out = architecture(inputs, random_rolls, True)
# with tf.Session() as sess:
#     writer = tf.summary.FileWriter('logs', sess.graph)
#     sess.run(tf.global_variables_initializer())
#     sess.run(out, feed_dict={inputs: X,
#                              random_rolls: random_rolls_batch})
#     writer.close()

In [8]:
# # confirm training = False gives same answers
# with tf.Session() as sess:
#     ls = []
#     with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
#         for i in range(3):
#             out = architecture(inputs, False)
#             if i == 0:
#                 sess.run(tf.global_variables_initializer())
#             ls.append(sess.run(out, feed_dict={inputs: X}))

In [9]:
# # confirm training = False but new init gives diff answers
# with tf.Session() as sess:
#     ls = []
#     with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
#         for i in range(3):
#             out = architecture(inputs, False)
#             sess.run(tf.global_variables_initializer())
#             ls.append(sess.run(out, feed_dict={inputs: X}))

In [10]:
# confirm training = True gives diff answers
# with tf.Session() as sess:
#     ls = []
#     with tf.variable_scope('stoch_depth', reuse=tf.AUTO_REUSE):
#         for i in range(3):
#             out = architecture(inputs, True)
#             if i == 0:
#                 sess.run(tf.global_variables_initializer())
#             ls.append(sess.run(out, feed_dict={inputs: X}))